In [7]:
from pyfinite import ffield
import math
import numpy as np

In [2]:
F = ffield.FField(5)


In [4]:
a = 7
F.ShowPolynomial(a) # show the polynomial representation of a

'x^2 + x^1 + 1'

In [9]:
def get_function_to_Zhegalkin_coeffs_matrix_G(n):
    if n in get_function_to_Zhegalkin_coeffs_matrix_G.cache:
        return get_function_to_Zhegalkin_coeffs_matrix_G.cache[n]
    
    G_n_prev = get_function_to_Zhegalkin_coeffs_matrix_G(n - 1)
    result = np.asmatrix(np.vstack([np.hstack([G_n_prev, np.zeros(G_n_prev.shape, dtype=int)]),\
                                    np.hstack([G_n_prev, G_n_prev])]))
    
    get_function_to_Zhegalkin_coeffs_matrix_G.cache[n] = result
    return result

get_function_to_Zhegalkin_coeffs_matrix_G.cache = {0: np.matrix([1], dtype=int)}

In [10]:
def get_positional_functions(permutation):
    n = round(math.log(len(permutation), 2))
    result = []
    for i in range(n):
        shift = n - i - 1
        another_function = [ (permutation[j] >> shift) & 1 for j in range(len(permutation)) ]
        result.append(np.matrix(another_function).T)
    return result

In [11]:
get_function_to_Zhegalkin_coeffs_matrix_G(2)

matrix([[1, 0, 0, 0],
        [1, 1, 0, 0],
        [1, 0, 1, 0],
        [1, 1, 1, 1]])

In [23]:
def str_Zhegalkin_polynomial(Zhegalkin_coeffs):
    n = round(math.log(len(Zhegalkin_coeffs), 2))
    included_conjunctions = []
    
    for i in range(1, len(Zhegalkin_coeffs)):
        if Zhegalkin_coeffs[i]:
            variable_vector = []
            included_variables = bin(i)[2:].zfill(n)
            for j in range(len(included_variables)):
                if included_variables[j] != '0':
                    variable_vector.append(j + 1)
            included_conjunctions.append(variable_vector)
    
    included_conjunctions.sort(key=lambda conj: (len(conj), conj))
    conjunctions_strings = [conjunction_to_str(conj) for conj in included_conjunctions]
    if Zhegalkin_coeffs[0]:
        conjunctions_strings = ['1'] + conjunctions_strings
    return ' ⊕ '.join(conjunctions_strings)

In [26]:
def conjunction_to_str(conjunction):
    """Конвертирует конъюнкцию, представляемую вектором входящих переменных, в строковую запись.
    [1, 3, 4] -> x1x3x4
    с нижними индексами в Libre Office."""
    
    return ''.join(map(lambda var: "x_{{{}}}".format(var), conjunction))  # Господи, питон, серьёзно?

In [63]:
def zhegalkin_polynomials_from_transform(permutation):
    n = round(math.log(len(permutation), 2))
    
    positional_functions = get_positional_functions(permutation)
    print(positional_functions)
    positional_functions_Zhegalkin_polynomials =\
        [(get_function_to_Zhegalkin_coeffs_matrix_G(n) * positional_function) % 2 for\
         positional_function in positional_functions]
    print(positional_functions_Zhegalkin_polynomials)
        
    positional_functions_Zhegalkin_polynomials = map(lambda matrix: matrix.A1,\
                                                     positional_functions_Zhegalkin_polynomials)
    positional_functions_Zhegalkin_polynomials = list(positional_functions_Zhegalkin_polynomials)
    
    Zhegalkin_polynomials_strings = map(str_Zhegalkin_polynomial, positional_functions_Zhegalkin_polynomials)
    Zhegalkin_polynomials_strings = list(Zhegalkin_polynomials_strings)
    print(Zhegalkin_polynomials_strings)
    
    for i in range(len(Zhegalkin_polynomials_strings)):
        Zhegalkin_polynomials_strings[i] = 'y_{{{}}} = '.format(i + 1) + Zhegalkin_polynomials_strings[i]
    return '; '.join(Zhegalkin_polynomials_strings)

In [64]:
zhegalkin_polynomials_from_transform([3,3,3,2])

[matrix([[1],
        [1],
        [1],
        [1]]), matrix([[1],
        [1],
        [1],
        [0]])]
[matrix([[1],
        [0],
        [0],
        [0]]), matrix([[1],
        [0],
        [0],
        [1]])]
['1', '1 ⊕ x_{1}x_{2}']


'y_{1} = 1; y_{2} = 1 ⊕ x_{1}x_{2}'

In [39]:
def mult(elem1, elem2, field):
    return field.Multiply(elem1, elem2)

In [40]:
def powF(elem, power, field):
    if power < 1 or isinstance(power, int) == False :
        raise Exception('power must be 1 or more and integer')
    res = elem
    for i in range(power - 1):
        res = mult(res, elem, field)
    return res
        

In [53]:
def getKloostermanApn(n=3):
    if n % 2 == 0:
        raise Exception('n must be odd')
    F = ffield.FField(n)
    power = 2 ** n - 2
    res_transform = []
    for i in range(2**n):
        res_transform.append(powF(i, power, F))
    return res_transform
    

In [54]:
powF(0,9,ffield.FField(2))

0

In [57]:
getKloostermanApn(3)

[0, 1, 5, 6, 7, 2, 3, 4]

In [58]:
def isPermutation(transform):
    return len(set(transform)) == len(transform)

In [62]:
isPermutation(getKloostermanApn(9))

True